In [14]:
import pickle
import os
import glob
import matplotlib.pyplot as plt
import pandas as pd

In [15]:
group_id = "default"

# load result
result_files = glob.glob(f"./results/{group_id}/*.pickle")
results = []
for file in result_files:
    with open(file, "rb") as f:
        results.append(pickle.load(f))

# create plots folder
os.makedirs(f"./plots/{group_id}/", exist_ok=True)


In [60]:
# make results into dataframe
df_columns = ["model", "label", "accuracy", "precision", "TP", "TN", "FP", "FN", "n_samples", "n_trials", "model"]
df_data = []
for r in results:
    args = r["args"]
    n_trials = args.n_trials
    max_retry = args.max_retry
    model = args.model
    for trial in r["result"]:
        for sample in trial:
            df_data.append([
                model,
                sample["class label"],
                sample["accuracy"],
                sample["precision"],
                sample["TP"],
                sample["TN"],
                sample["FP"],
                sample["FN"],
                sample["n_samples"],
                n_trials,
                model,
            ])
df_results = pd.DataFrame(df_data, columns=df_columns)
df_results

,model,label,accuracy,precision,TP,TN,FP,FN,n_samples,n_trials,model
0,gpt-3.5-turbo,Company,1.0,1.0,5,5,0,0,5,1,gpt-3.5-turbo
1,gpt-3.5-turbo,EducationalInstitution,0.7,0.4,2,5,3,0,5,1,gpt-3.5-turbo
2,gpt-3.5-turbo,Artist,0.7,0.4,2,5,3,0,5,1,gpt-3.5-turbo
3,gpt-3.5-turbo,Athlete,0.3,0.0,0,3,5,2,5,1,gpt-3.5-turbo
4,gpt-3.5-turbo,OfficeHolder,0.8,1.0,5,3,0,2,5,1,gpt-3.5-turbo
5,gpt-3.5-turbo,MeanOfTransportation,0.7,1.0,5,2,0,3,5,1,gpt-3.5-turbo
6,gpt-3.5-turbo,Building,0.8,0.6,3,5,2,0,5,1,gpt-3.5-turbo
7,gpt-3.5-turbo,NaturalPlace,0.6,0.2,1,5,4,0,5,1,gpt-3.5-turbo
8,gpt-3.5-turbo,Village,0.5,0.0,0,5,5,0,5,1,gpt-3.5-turbo
9,gpt-3.5-turbo,Animal,0.6,0.8,4,2,1,3,5,1,gpt-3.5-turbo


In [61]:
# group by n_sample
res_g_sample = df_results.groupby(by=["label","n_samples"], as_index=False).mean("precision", "accuracy")
res_g_sample

,label,n_samples,accuracy,precision,TP,TN,FP,FN,n_trials
0,Album,5,0.9,0.8,4.0,5.0,1.0,0.0,1.0
1,Animal,5,0.6,0.8,4.0,2.0,1.0,3.0,1.0
2,Artist,5,0.7,0.4,2.0,5.0,3.0,0.0,1.0
3,Athlete,5,0.3,0.0,0.0,3.0,5.0,2.0,1.0
4,Building,5,0.8,0.6,3.0,5.0,2.0,0.0,1.0
5,Company,5,1.0,1.0,5.0,5.0,0.0,0.0,1.0
6,EducationalInstitution,5,0.7,0.4,2.0,5.0,3.0,0.0,1.0
7,Film,5,1.0,1.0,5.0,5.0,0.0,0.0,1.0
8,MeanOfTransportation,5,0.7,1.0,5.0,2.0,0.0,3.0,1.0
9,NaturalPlace,5,0.6,0.2,1.0,5.0,4.0,0.0,1.0


In [27]:
# plot accuracy and precision with df_results
# x axis: label
# y axis: accuracy/precision (average of all trials)
# group by n_sample

labels = {}

for _, row in df_results.iterrows():
    label = row["label"]
    if label not in labels:
        labels[label] = []
    labels[label].append(row)

# get average of accuracy and precision for each label group by n_sample
plots = {}
for label in labels:
    plots[label] = {}
    for n_sample in df_results.groupby(["n_samples"]).mean():
        plots[label][n_sample] = {
            "accuracy": sum([x["accuracy"] for x in labels[label] if x["n_samples"] == n_sample]) / len([x["accuracy"] for x in labels[label] if x["n_samples"] == n_sample]),
            "precision": sum([x["precision"] for x in labels[label] if x["n_samples"] == n_sample]) / len([x["precision"] for x in labels[label] if x["n_samples"] == n_sample]),
        }
# plot
plt.figure(figsize=(20, 5))
plt.title(f"{model} - {n_sample} samples - {n_trials} trials")
plt.xlabel("label")
plt.ylabel("accuracy/precision")
plt.ylim(0, 1)
plt.scatter(plots.keys(), [x["accuracy"] for x in plots.values()], label="accuracy")
plt.scatter(plots.keys(), [x["precision"] for x in plots.values()], label="precision")
plt.legend()
# save images
plt.savefig(f"./plots/{group_id}/{model}-{n_sample}-{n_trials}.png")


AttributeError: 'DataFrame' object has no attribute 'unique'